In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import nd2reader
import numpy as np
from cytoolz import compose
import skimage
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from paulssonlab.projects.molecule_counting.segmentation import segment, invert
from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels

# import cv2
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import signal
import time

mpl.rcParams["figure.figsize"] = (10, 10)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
# Set up timers so that functions timeout and move on if something is weird and they don't run properly.
# Should diagnose this issue properly eventually, but for testing this should be ok.
class TimeoutException(Exception):  # Custom exception class
    pass


def timeout_handler(signum, frame):  # Custom signal handler
    raise TimeoutException


# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch2/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[5]
print("file", file)
nd2 = nd2reader.ND2Reader(file)
fov_index = 0
# phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
# fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
phase_img = nd2.get_frame_2D(v=0, c=0, z=0)[1100:3000, 500:2250]
fluor_img = nd2.get_frame_2D(v=0, c=1, z=0)[1100:3000, 500:2250]
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img)
ax[1].imshow(fluor_img, cmap="RdGy")

In [ ]:
seg = segment_phase(phase_img)
fig, ax = plt.subplots()
ax.imshow(seg)

In [ ]:
def get_indices_in_r(ref_indices, r):
    # ref_indices is a list of the indices that define the location of the reference point.
    # r is the radius within which we want to collect indices.
    ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
    rel_points = np.where(
        (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
    )
    points = [
        rel_points[0] + ref_indices[0] - np.ceil(r),
        rel_points[1] + ref_indices[1] - np.ceil(r),
    ]
    return points

In [ ]:
def rdf(
    phase_img,
    fluor_img,
    cell_indices=[0],
    time_allowed=80,
    filename="20200603rdf",
    start_r=1,
    dr=1,
):
    # Segment the image.
    seg = segment_phase(phase_img)
    #     Get the bakground fluorescence of the image, to subtract from all cell fluorescence.
    min_fluor = np.amin(fluor_img)
    # Get number of cells in the image.
    cells = np.amax(seg)
    # all_r_values will be a list of lists of r values, while all_gr_values will be a list of lists, with
    # the corresponding g(r) (pair correlation function) values. The overall function will return
    # all_r_values, all_gr_values.
    all_r_values = []
    all_gr_values = []
    for cell_index in cell_indices:
        if cell_index > cells:
            print("Error in rdf: cell_index larger than number of cells in fov.")
            pass
        #         Mask the fluorescence image everywhere except for the cell.
        cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
        # Normalise the intensities of the cell.
        cell = (cell - min_fluor) / np.mean(cell)
        #         ref_point is the 'nucleus' i.e. the point relative to which the pair
        #         correlation function is to be calculated. For now, we set it to the
        #         maximum fluorescence intensity pint of the cell.
        ref_point = np.where(cell == np.amax(cell))
        #         If there is more than one maximum fluorescence point, take the first one of these.
        if len(ref_point[0]) > 1:
            ref_point = (ref_point[0][0], ref_point[1][0])
        # rho is the average 'particle' density, where a particle corresponds to an intensity unit.
        rho = np.mean(cell)
        #         print('rho',rho)
        r = start_r
        r_values = []
        #         cell.mask is False where the cell is, and True outside.
        mask = np.copy(cell.mask)
        gr_values = []
        previous_r_vol = 0
        while np.count_nonzero(cell.mask == 0) > 0:
            #             While the cell mask still has 'False' values i.e. not all values have been used in the
            #             pair correlation function calculation somewhere.
            #             Get indices of the points within the current radius (r) of the ref point.
            indices_in_r = get_indices_in_r(ref_point, r)
            flat_indices_in_r = [item for sublist in indices_in_r for item in sublist]
            img_shape = np.shape(fluor_img)
            if any(flat_indices_in_r >= np.min(img_shape)):
                indices_to_delete_0 = np.where(indices_in_r[0] >= img_shape[0])[0]
                indices_to_delete_1 = np.where(indices_in_r[1] >= img_shape[1])[0]
                indices_to_delete = np.concatenate(
                    (indices_to_delete_0, indices_to_delete_1)
                )
                indices_to_delete = np.unique(indices_to_delete)
                indices_in_r[0] = np.delete(indices_in_r[0], indices_to_delete)
                indices_in_r[1] = np.delete(indices_in_r[1], indices_to_delete)
            #                 indices_in_r[0] = np.delete(indices_in_r[0], np.where(indices_in_r[0] >= img_shape[0]))
            #                 indices_in_r[1] = np.delete(indices_in_r[1], np.where(indices_in_r[1] >= img_shape[1]))

            #                     if indices_in_r[0][i]>=img_shape[0] or indices_in_r[1][i]>=img_shape[1]:
            #                         print('deleting0')
            #                         del indices_in_r[0][index]
            #                         del indices_in_r[1][index]
            #             print('indicesinr',indices_in_r)
            # Calculate numerator (gr_num) and volume element (gr_vol)
            # gr_num should be the sum total of intensities of points in the current shell.
            #             Since cell is masked, we don't need to worry about the non-cell regions.
            #             print(cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))])
            gr_num = np.sum(
                cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
            )
            #             print('gr num',cell_index,gr_num)
            #             gr_num_all and gr diff are just to check that the cell masking is working.
            gr_num_all = np.sum(
                cell.data[
                    list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
                ]
            )
            #             print('gr num all',cell_index,gr_num_all)
            #             print('gr diff',cell_index,gr_num_all-gr_num)
            # gr_vol should be the number of points in the current shell.
            #             Note that the mask masks both the background and the previous regions that we have considered,
            #             so we simply subtract these from our list of indices in the current radius.
            gr_vol = len(indices_in_r[0]) - np.count_nonzero(
                cell.mask[
                    list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
                ]
            )
            #             update the 'previous_r_vol' for the next radius shell.
            #             print('previous r vol',cell_index,previous_r_vol)
            previous_r_vol = gr_vol
            #             print('gr vol',cell_index,gr_vol)
            #             Update the cell mask so that the values we have now included are not included
            #             in the next dr calculation.
            cell.mask[
                list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
            ] = True
            #             print('number of non masked values',np.count_nonzero(cell.mask==0))
            gr_values.append(gr_num / gr_vol)
            r_values.append(r)
            r += dr
        # Since rho is constant for all r, divide all at the same time to save on number of computations.
        gr_values = np.array(gr_values)
        #         print('gr_values beofre rho',gr_values)
        gr_values = np.divide(gr_values, rho)
        #         print('gr_values divided by rho',gr_values)
        all_gr_values.append(gr_values)
        all_r_values.append(r_values)
    return all_r_values, all_gr_values

In [ ]:
# Test using seg as the fluor image, since this should return g(r) = 1 for all r.
# r_values_list, gr_values_list = rdf(phase_img,seg,cell_indices=np.arange(6),dr=10)
r_values_list, gr_values_list = rdf(
    phase_img, fluor_img, cell_indices=np.arange(20), dr=1
)

In [ ]:
fig, ax = plt.subplots()
for i in range(len(r_values_list)):
    ax.plot(r_values_list[i], gr_values_list[i])
ax.axhline(y=1, ls=":", color="k")
fig.savefig("20200613_P_GFPmut3_40x")

In [ ]:
def get_stdevs_alpha_time_cellnorm(
    img_file, fov_indices, time_allowed=80, savetxt=False, filename=None
):
    # Normalises each image. Subtract background (non-cell) and
    # then divide by the mean fluorescence in the specific cell.
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    fluor_diffs = []
    for fov_index in fov_indices:
        signal.alarm(time_allowed)
        try:
            print(str(img_file), "fov index", fov_index)
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            # Segment the image.
            seg = segment_phase(phase_img)
            min_fluor = np.amin(fluor_img)
            # Get number of cells in the image.
            cells = np.amax(seg)
            # Get the standard deviation in fluorescence of each cell in the image.
            for i in range(cells):
                cell = np.ma.masked_where(seg != i + 1, fluor_img)
                cell = (cell - min_fluor) / np.mean(cell)
                stdevs_alpha.append(np.std(cell))
                fluor_diffs.append(np.amax(cell) - np.amin(cell))
            #                 print(np.amin(cell),np.amax(cell), np.amax(cell)-np.amin(cell),np.std(cell))
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
        except TimeoutException:
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
            print("timeout")
            continue  # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    return stdevs_alpha

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(30, 15), sharex=True)
ax = axes.ravel()
logmeans = []
logstds = []
for i in range(len(files_names)):
    try:
        data_stdevs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_alpha_cellnorm.csv"
        )
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        #         ax[i].hist(data_fluordiffs, bins = 50)
        ax[i].set_title(files_names[i])
        plt.xscale("log")
        print(files_names[i], "number of cells:", len(data_stdevs))
        print(files_names[i], "mean of stdevs:", np.mean(data_stdevs))
        print(files_names[i], "stdev of stdevs:", np.std(data_stdevs))
        print(
            files_names[i],
            "mean of log10 stdevs:",
            np.mean(np.log10(data_stdevs)),
            10 ** (np.mean(np.log10(data_stdevs))),
        )
        logmeans.append(10 ** (np.mean(np.log10(data_stdevs))))
        print(
            files_names[i],
            "stdev of log10 stdevs:",
            np.std(np.log10(data_stdevs)),
            10 ** (np.std(np.log10(data_stdevs))),
        )
        logstds.append(10 ** (np.std(np.log10(data_stdevs))))
    except OSError:
        pass

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
ax = axes.ravel()
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[0].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[0].plot(logmeans[i], logstds[i], "ro")
ax[0].axhline(1.7, linestyle="--", color="k")
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[1].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[1].plot(logmeans[i], logstds[i], "ro")
ax[1].axhline(1.7, linestyle="--", color="k")
for i, txt in enumerate(files_names):
    ax[1].annotate(txt, (logmeans[i] + 0.001, logstds[i] + 0.05))
plt.xlabel("Mean of cell stdevs")
plt.ylabel("Stdev of cell stdevs")

In [ ]:
for i in range(len(files_names)):
    if logstds[i] < 1.7:
        print(files_names[i])

In [ ]:
files_names = [
    "chr_P_sfGFP",
    "chr_P_GFPmut3",
    "chr_P_Venus",
    "chr_P_Dendra2",
    "plas_X_DHL_GFPmut3",
    "plas_P_DHL_sfGFP40to54",
    "plas_X_arai_sfGFP",
    "chr_P_GFPmut3_40x",
    "chr_P_sfGFP_40x",
    "chr_P_Venus_40x",
    "chr_P_Dendra2_40x",
    "chr_P_sfGFP_20x",
    "chr_P_GFPmut3_20x",
    "chr_P_Dendra2_20x",
    "chr_P_Venus_20x",
    "plas_X_DHL_GFPmut3_20x",
    "plas_X_DHL_GFPmut3_40x",
    "plas_P_DHL_sfGFP_20x",
    "plas_P_DHL_sfGFP_40x",
]

fig, axes = plt.subplots(3, 1, figsize=(3, 9), sharex=True)
ax = axes.ravel()
# names = ['chr_P_sfGFP', 'chr_P_sfGFP_40x', 'chr_P_sfGFP_20x']
# names = ['chr_P_GFPmut3', 'chr_P_GFPmut3_40x', 'chr_P_GFPmut3_20x']
# names = ['chr_P_Venus', 'chr_P_Venus_40x', 'chr_P_Venus_20x']
# names = ['chr_P_Dendra2', 'chr_P_Dendra2_40x', 'chr_P_Dendra2_20x']
# names = ['plas_X_DHL_GFPmut3', 'plas_X_DHL_GFPmut3_40x', 'plas_X_DHL_GFPmut3_20x']
# names = ['plas_P_DHL_sfGFP40to54', 'plas_P_DHL_sfGFP_40x', 'plas_P_DHL_sfGFP_20x']

base = names[1][:-4]
ax[0].set_title(base + " 100x")
ax[1].set_title(base + " 40x")
ax[2].set_title(base + " 20x")

for i in range(len(names)):
    try:
        data_stdevs = np.loadtxt("202004_cellnorm/" + names[i] + "_alpha_cellnorm.csv")
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        plt.xscale("log")
    except OSError:
        pass
fig.savefig(base)